In [1]:
#This notebook summarizes uploaded PDFs using a Hugging Face transformer model, no API key required.

In [2]:
try:
    import transformers
    import sentence_transformers
    import PyPDF2  # Kept for PDF reading
    import torch  # Kept for tensor operations
    print("Libraries are already installed.")
except ImportError:
    print("Installing libraries...")
    !pip install transformers sentence-transformers PyPDF2 torch

from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from PyPDF2 import PdfReader
import os  # Necessary for environment variable setting

Libraries are already installed.


In [3]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
reader = PdfReader(pdf_path)
text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

Saving Justin Wilson Bio.pdf to Justin Wilson Bio (2).pdf


In [4]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i+chunk_size])
    return chunks

chunks = chunk_text(text)

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
def search_chunks(question, chunks, embeddings, model, top_k=3):
    question_embedding = model.encode(question, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(question_embedding, embeddings)[0]
    top_results = torch.topk(scores, k=top_k)
    top_chunks = [chunks[i] for i in top_results[1]]
    return "\n\n".join(top_chunks)

question = "What are the key takeaways?"
relevant_text = search_chunks(question, chunks, embeddings, model)

In [6]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(
    "Summarize the following into 5–7 key bullet points:\n\n" + relevant_text,
    max_length=512,
    min_length=100,
    do_sample=False
)

Device set to use cpu
Your max_length is set to 512, but your input_length is only 267. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=133)


In [7]:
# Split the summary text into bullet points using sentence ends
summary_text = summary[0]['summary_text']
sentences = summary_text.split(". ")

# Take first 5–7 non-empty sentences and format as bullets
bulleted_points = [f"• {s.strip()}." for s in sentences if s.strip()][:7]

# Print each bullet point
for point in bulleted_points:
    print(point)

• Justin may humbly lessen the significance of his accomplishments, but his greatest pride will always be his family.
• Justin’s greatest hope is that he deepens the legacy lovingly passed down to him.
• The Wilson family has appeared on numerous commercials and ad campaigns.
• They’ve partnered with companies some of which are Google, Apple, Disney, Home Depot, Meta, HBO, Amazon, McDonalds, and Walmart to name a few.
• With his family, Justin has become an expert in managing media campaigns and crafting compelling stories..
